<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/number_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [276]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [23]:
# Define transforms to convert PIL images to tensors
transform = transforms.ToTensor()

# Download and load MNIST training data
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Download and load MNIST test data
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [854]:
dl_train = DataLoader(mnist_train, batch_size=16, shuffle=True)
dl_valid = DataLoader(mnist_test, batch_size=16)
xb, yb = next(iter(dl_train))
xb.shape, yb.shape
xb = xb.view(xb.size(0), -1)
#yb = yb.view(yb.size(0), -1)
xb.shape,yb.shape
xb[0], yb[0], yb.shape
# preds = linear1(xb[:4])
# loss = mnist_loss(preds, yb[:4])
# loss



(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [919]:
class BasicOptimizer:
  def __init__(self, params, learnrate):
    self.params = list(params)
    self.learnrate = learnrate

  def step(self, *args, **kwargs):
    for p in self.params:
      #p.data -= p.grad.data * self.learnrate #using .data so it isn't calculated into gradient
      if p.grad is not None:
            with torch.no_grad():
                p.sub_(self.learnrate * p.grad)
                print(p.grad)
  def zero_grad(self, *args, **kwargs):
    for p in self.params:
      if p.grad is not None:
        p.grad.zero_()

In [808]:
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()

# own softmax function for practice, use torch.softmax(logits, dim=1)
# subtracting x.max(dim=1).values for numerical stability. Dividing by e^{-m} / e^{-m}
def softmax(x):
  e_x = torch.exp(x - x.max(dim=1, keepdim=True).values)
  return e_x / e_x.sum(dim = 1, keepdim=True)
def mnist_loss(prediction, target):
  eps = 1e-7
  prediction = softmax(prediction)
  prediction = prediction.clamp(min=eps)
  idx = range(target.size(0))
  prediction = prediction[idx, target]
  # print("This is prediction", prediction)
  # print("THis is log torch: ", -torch.log(prediction))
  return (-torch.log(prediction)).mean();
def calc_grad(xb, yb, model):
  preds = model(xb)
  print(preds)
  loss = mnist_loss(preds, yb)
  loss.backward()
  print("This is loss: ", loss)
  print("w1.grad: ", w1.grad)

In [857]:
# need 7840 weights. Have "10 additional neurons in the second layer"
w1, w2 = init_params((784, 16)), init_params((16, 10))
b1, b2 = init_params(16), init_params(10)
# weights = weights.to(device)
# bias = bias.to(device)
def linear1(x):
  x = x@w1 + b1
  x = F.relu(x)
  x = x@w2 + b2
  return x
w1, w2, b1, b2

(tensor([[ 0.2087,  0.2708, -0.2946,  ...,  1.2612,  0.1097, -2.1936],
         [ 0.2355, -0.2921,  2.5281,  ..., -1.2146, -0.9865, -0.0290],
         [-1.8891, -0.6420, -1.0694,  ..., -0.1945, -0.1685, -0.5146],
         ...,
         [ 0.6260,  0.8259, -1.1655,  ...,  0.3345, -0.3178, -0.2549],
         [-0.1475,  0.7359, -1.1288,  ...,  0.6662,  0.4006, -0.3043],
         [-1.0513,  2.0267, -1.6971,  ...,  1.2545, -1.3512,  0.5722]],
        requires_grad=True),
 tensor([[ 2.2606, -0.2366, -0.8782, -1.8847,  0.4226,  0.2594, -0.0603, -0.5850,
          -1.3004,  0.1785],
         [-0.0312,  0.6892,  0.1988, -1.3463, -0.2005, -0.6303,  1.3157, -1.5337,
           0.2854, -1.3371],
         [-0.6638, -0.1190,  0.7257, -0.2490, -0.2130,  0.0561, -3.1774, -0.9594,
          -0.3486,  0.3311],
         [ 0.6473,  1.0024,  0.8935, -1.4156, -0.1087, -0.2503,  0.1463, -0.0999,
          -0.7821, -1.2600],
         [ 0.0118,  0.8522, -0.6198,  0.5935, -0.7837, -0.9077, -0.1227, -0.4979,
    

In [772]:
def train_epoch(model):
  for xb, yb in dl_train:
    xb = xb.view(xb.size(0), -1)
    yb = yb.view(-1)
    calc_grad(xb, yb, model)
    print("w1[0,0] before step:", w1[0,0].item())
    opt.step()
    print("w1[0,0] after step:", w1[0,0].item())
    opt.zero_grad()

def batch_accuracy(preds, yb): # xb is pure logits
  _,preds_class = torch.max(preds, dim=1)
  return (preds_class == yb).float().mean()

def validate_epoch(model):
  accs = [batch_accuracy(model(xb.view(xb.size(0), -1)), yb.view(yb.size(0), -1)) for xb, yb in dl_valid]
  return round(torch.stack(accs).mean().item(), 4)

def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end=' ')

In [921]:
opt = BasicOptimizer((w1, w2, b1, b2), 0.5)
#train_epoch(linear1)
xb, yb = next(iter(dl_train))
xb.shape,yb.shape
xb = xb.view(xb.size(0), -1)
# train_model(linear1, 5)
out = linear1(xb).sum()
print("Sum: ", out)
out.backward()
print("w1.grad: ", b2.grad)
if w1.grad is not None:
    print("JOEMAMMAAM")
opt.step()
opt.zero_grad()
print("w1.grad: ", b1.grad)
#calc_grad(xb, yb, linear1)
# opt.step()
# opt.zero_grad()
# preds = linear1(xb)
# loss = mnist_loss(preds, yb)
# loss, yb.shape, preds, softmax(preds)
# print("Loss: ", loss)
# loss.backward()
#calc_grad(xb, yb, linear1)
#print("In optimizer: this is weights ", w2.grad)
#print("loss after: ", loss)

Sum:  tensor(-inf, grad_fn=<SumBackward0>)
w1.grad:  tensor([16., 16., 16., 16., 16., 16., 16., 16., 16., 16.])
JOEMAMMAAM
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[9.0336e+34, 9.0336e+34, 9.0336e+34, 9.0336e+34, 9.0336e+34, 9.0336e+34,
         9.0336e+34, 9.0336e+34, 9.0336e+34, 9.0336e+34],
        [3.7180e+36, 3.7180e+36, 3.7180e+36, 3.7180e+36, 3.7180e+36, 3.7180e+36,
         3.7180e+36, 3.7180e+36, 3.7180e+36, 3.7180e+36],
        [2.0372e+36, 2.0372e+36, 2.0372e+36, 2.0372e+36, 2.0372e+36, 2.0372e+36,
         2.0372e+36, 2.0372e+36, 2.0372e+36, 2.0372e+36],
        [1.5412e+36, 1.5412e+36, 1.5412e+36, 1.5412e+36, 1.5412e+36, 1.5412e+36,
         1.5412e+36, 1.5412e+36, 1.5412e+36, 1.5412e+36],
        [4.6340e+36, 4.6340e+36, 4.6340e+36, 4.6340e+36, 4.6340e+36

In [63]:
accs = validate_epoch(linear1)
accs
train_model(linear1, 20)

0.098 0.098 0.098 

KeyboardInterrupt: 

In [682]:
model = nn.Sequential(
    nn.Linear(28*28, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
).to(device)

# STEP 5: Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# STEP 6: Accuracy helper
def accuracy(model, dataloader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.view(xb.size(0), -1)
            preds = model(xb)
            pred_labels = preds.argmax(dim=1)
            correct += (pred_labels == yb).sum().item()
            total += yb.size(0)
    return correct / total

# STEP 7: Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    for xb, yb in dl_train:
        xb, yb = xb.to(device), yb.to(device)
        xb = xb.view(xb.size(0), -1)  # Flatten the image

        preds = model(xb)
        loss = loss_fn(preds, yb)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    val_acc = accuracy(model, dl_valid)
    print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_acc:.4f}")

Epoch 1/5, Validation Accuracy: 0.9062
Epoch 2/5, Validation Accuracy: 0.9156
Epoch 3/5, Validation Accuracy: 0.9216
Epoch 4/5, Validation Accuracy: 0.9279
Epoch 5/5, Validation Accuracy: 0.9310
